In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import random 

In [2]:
data = pd.read_csv('Exercises.csv')

In [3]:
data

,Age,Height,Weight,Gender,injury,equipment,level,Exercise
0,20,175,70,male,Upper,Body Only,Beginner,Arom Side lying Abduction
1,21,175,71,male,Lower,Body Only,Beginner,Air Squat
2,22,175,72,male,Abdominals & Lower Back,Body Only,Intermediate,Dead Bug
3,23,175,73,male,Upper,Dumbbells,Intermediate,Dumbbell Front Raise
4,24,175,74,male,Lower,Dumbbells,Intermediate,Dumbbell Squat
...,...,...,...,...,...,...,...,...
9853,44,191,101,male,Upper,Dumbbells,Intermediate,Bent-Over Dumbbell Rear Delt Row
9854,45,175,101,male,Upper,Dumbbells,Intermediate,Cuban Press
9855,46,176,101,male,Lower,Dumbbells,Intermediate,Dumbbell Deadlift
9856,37,184,101,male,Lower,Dumbbells,Intermediate,Dumbbell Front Squat


In [4]:
# Mapping gender to 0 and 1
data['gender_encoded'] = data['Gender'].map({'male': 0, 'female': 1})

In [5]:
X = data[['Age', 'Height', 'Weight',  'injury','gender_encoded']]
y = data['Exercise']

In [6]:
X_encoded = pd.get_dummies(X, columns=['injury'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [8]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
y_pred = rf_classifier.predict(X_test)

In [10]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.82


In [11]:

# Function to predict exercise
def predict_exercise(model, user_input, X_encoded):
    # Prepare the user input for prediction
    user_input['gender_encoded'] = 1 if user_input['gender'].lower() == 'female' else 0
    user_input_df = pd.DataFrame([user_input])
    user_input_encoded = pd.get_dummies(user_input_df, columns=['injury'])
    user_input_encoded = user_input_encoded.reindex(columns=X_encoded.columns, fill_value=0)
    
    # Predict exercise
    predicted_exercise = model.predict(user_input_encoded)[0]
    return predicted_exercise

In [12]:
# Function to filter exercises based on user specifications
def filter_exercises(data, injury, equipment, level):
    # Adjust equipment filter
    if equipment.lower() == 'dumbbells':
        equipment_filter = ['Dumbbells', 'Body Only']
    else:
        equipment_filter = [equipment.lower()]
    
    # Adjust level filter
    if level.lower() == 'beginner':
        level_filter = ['Beginner']
    elif level.lower() == 'intermediate':
        level_filter = ['Beginner', 'Intermediate']
    elif level.lower() == 'advanced':
        level_filter = ['Beginner', 'Intermediate', 'Advanced']
    else:
        level_filter = [level.lower()]
    
    filtered_df = data[(data['injury'] == injury) & (data['equipment'].isin(equipment_filter)) & (data['level'].isin(level_filter))]
    return filtered_df


In [13]:
# Main function to get recommendation
def get_exercise_recommendation(user_input, model, data, X_encoded, n=3):
    predicted_exercise = predict_exercise(model, user_input, X_encoded)
    
    # Check if the predicted exercise matches the user's specifications
    predicted_exercise_row = data[data['Exercise'] == predicted_exercise]
    
    if not predicted_exercise_row.empty:
        predicted_exercise_matches = (
            (predicted_exercise_row['equipment'].values[0].lower() in ['dumbbells', 'body only']) if user_input['equipment'].lower() == 'dumbbells' else (predicted_exercise_row['equipment'].values[0].lower() == user_input['equipment'].lower()) and
            (predicted_exercise_row['level'].values[0].lower() in ['beginner', 'intermediate', 'advanced']) if user_input['level'].lower() == 'advanced' else (predicted_exercise_row['level'].values[0].lower() in ['beginner', 'intermediate']) if user_input['level'].lower() == 'intermediate' else (predicted_exercise_row['level'].values[0].lower() == user_input['level'].lower())
        )
        
        if predicted_exercise_matches:
            initial_recommendations = [predicted_exercise]
        else:
            initial_recommendations = []
    else:
        initial_recommendations = []
    
    # Filter exercises based on user's injury, equipment, and level
    filtered_exercises = filter_exercises(data, user_input['injury'], user_input['equipment'], user_input['level'])
    
    # Get unique exercises from the filtered list
    unique_exercises = list(filtered_exercises['Exercise'].unique())
    
    # Initialize recommendations with predicted exercise if it matches the criteria
    recommendations = list(set(initial_recommendations))
    
    # Add additional unique exercises to meet the required number
    for exercise in unique_exercises:
        if exercise not in recommendations:
            recommendations.append(exercise)
        if len(recommendations) >= n:
            break
    
    # If there are not enough unique recommendations, randomly sample more from the filtered list
    if len(recommendations) < n:
        remaining_exercises = list(filtered_exercises['Exercise'])
        random.shuffle(remaining_exercises)
        for exercise in remaining_exercises:
            if exercise not in recommendations:
                recommendations.append(exercise)
            if len(recommendations) >= n:
                break
    
    # If we still don't have enough exercises, expand the search criteria
    if len(recommendations) < n:
        # Fallback: Get random exercises from the same injury category
        fallback_exercises = data[data['injury'] == user_input['injury']]['Exercise'].unique()
        random.shuffle(fallback_exercises)
        for exercise in fallback_exercises:
            if exercise not in recommendations:
                recommendations.append(exercise)
            if len(recommendations) >= n:
                break

    return recommendations[:n]

In [14]:
# Example user input
user_input = {
    'Age': 20,
    'Height': 175,
    'Weight': 70,
    'gender': 'male',
    'injury': 'Upper',
    'equipment': 'body only',
    'level': 'Beginner'
}

In [15]:
# Get exercise recommendation
recommended_exercises = get_exercise_recommendation(user_input, rf_classifier, data, X_encoded, n=5)

print(recommended_exercises)

['Arom Side lying Abduction', 'Dumbbell Front Raise', 'Dumbbell Scaption', 'Prone Horizontal Abduction', 'Bent-Over Dumbbell Rear Delt Row']


In [20]:
import joblib
joblib.dump(rf_classifier, 'model.pkl' ,compress = 3)

['model.pkl']